In [1]:
# ~ The Connection ~ #

import pandas as pd
import psycopg2

#establishing the connection
conn = psycopg2.connect(
    database="puptrail", 
    user='dw_chancewiese',
    password='Spikeball2020',
    host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
    port='5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("select version()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)


Connection established to:  ('PostgreSQL 16.3 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-6), 64-bit',)


In [3]:
# Table Creation! #

# Customer Table
query = '''CREATE TABLE IF NOT EXISTS customer (
            custid INT PRIMARY KEY,
            custfname VARCHAR(100) NOT NULL,
            custlname VARCHAR(100) NOT NULL,
            custphone VARCHAR(14),
            custemail VARCHAR(255),
            custstate CHAR(2),
            contactpref VARCHAR(10),
            pmtid VARCHAR(20),
            addedstamp TIMESTAMP NOT NULL,
            updatedstamp TIMESTAMP,
            hhi INT,
            ismarried CHAR(1),
            haskids CHAR(1),
            travelswpet CHAR(1),
            pronoun VARCHAR(20)
        );'''

cursor.execute(query)
conn.commit()


In [5]:
# Oline Table
query = '''CREATE TABLE IF NOT EXISTS oline (
            olid SERIAL PRIMARY KEY,
            orderid INT,
            productid INT NOT NULL,
            lineqty SMALLINT NOT NULL,
            lineprice NUMERIC(10,2) NOT NULL,
            linetotal NUMERIC(10,2) NOT NULL,
            builtdate DATE,
            promisedate DATE,
            deldate DATE)'''

cursor.execute(query)
conn.commit()

In [7]:
# Pricelog Table
query = '''CREATE TABLE IF NOT EXISTS pricelog (
        priceid SERIAL PRIMARY KEY,
        prodid INT NOT NULL,
        theprice NUMERIC(10,2) NOT NULL,
        startdate DATE,
        approver VARCHAR(50),
        approvedate DATE)
        '''

cursor.execute(query)
conn.commit()

In [9]:
# Prodcat Table
query = '''CREATE TABLE IF NOT EXISTS prodcat(
        catid INT PRIMARY KEY,
        category VARCHAR(50) NOT NULL,
        catdesc TEXT)'''

cursor.execute(query)
conn.commit()

In [11]:
# product table
query = '''CREATE TABLE IF NOT EXISTS product(
        prodid INT PRIMARY KEY,
        catid INT,
        sourceid INT,
        prodname VARCHAR(100) NOT NULL,
        msrp NUMERIC(10,2),
        proddesc TEXT,
        leadtime SMALLINT,
        qtyoh SMALLINT,
        startdate DATE,
        enddate DATE,
        rolevel SMALLINT)'''

cursor.execute(query)
conn.commit()

In [13]:
#SALESORDER TABLE
query = '''CREATE TABLE IF NOT EXISTS salesorder (
        orderid INT PRIMARY KEY,
        custid INT NOT NULL,
        sellerid INT,
        odate DATE NOT NULL,
        ototal NUMERIC(10,2),
        ordsatisfac SMALLINT,
        delsatisfac SMALLINT)'''

cursor.execute(query)
conn.commit()

In [15]:
#Salesperson Table
query = '''CREATE TABLE IF NOT EXISTS salesperson (
        spid INT PRIMARY KEY,
        siteid CHAR(2),
        spfname VARCHAR(50) NOT NULL,
        splname VARCHAR(50) NOT NULL,
        startdate DATE,
        termdate DATE)'''

cursor.execute(query)
conn.commit()

In [17]:
#Site Table
query = '''CREATE TABLE IF NOT EXISTS site (
        siteid CHAR(2) PRIMARY KEY,
        sitename VARCHAR(50) NOT NULL)'''

cursor.execute(query)
conn.commit()

In [19]:
#Vendor Table
query = '''CREATE TABLE IF NOT EXISTS vendor (
        vendorid SERIAL PRIMARY KEY,
        vendorname VARCHAR(100) NOT NULL,
        contactname VARCHAR(100))'''

cursor.execute(query)
conn.commit()

In [21]:
# Populating!!! #

# Site Table

import pandas as pd

df = pd.read_csv('site.csv')
print(df)

for x in df.index:
    
    cursor.execute("""
        INSERT INTO Site (siteid, sitename)
        VALUES ('%s', '%s')
    """ % (df.loc[x]['SiteID'],
          df.loc[x]['SiteName']))
    
#    query = query + "'" + df.loc[x]['CorgName'] ');'"
    
    conn.commit()

cursor.execute("""SELECT * FROM site""")

for x in cursor:
    print(x)

  SiteID        SiteName
0     SL  Salt Lake City
1     BO           Boise
2     PH         Phoenix
('SL', 'Salt Lake City')
('BO', 'Boise')
('PH', 'Phoenix')


In [23]:
# The Salesperson Table
# Make sure there's no SPID in the .csv before loading! *Or maybe not...*

import psycopg2
import numpy as np
#conn.close()


df = pd.read_csv('salesperson.csv')
df.replace(to_replace=np.nan, value='NULL', inplace=True)
print(df)

for x in df.index:
    
    if df.loc[x]['TermDate'] == 'NULL':
        termdate = 'NULL'
    else:
        termdate = "'"+df.loc[x]['TermDate']+"'"
    
    cursor.execute("""
        INSERT INTO salesperson (spid,siteid,spfname,splname,startdate,termdate)
        VALUES (%s, '%s','%s','%s','%s',%s)
    """ % (df.loc[x]['SPID'],
          df.loc[x]['SiteID'],
          df.loc[x]['Spfname'],
          df.loc[x]['Splname'],
          df.loc[x]['StartDate'],
          termdate))
    
    conn.commit()
    
cursor.execute("""SELECT * FROM salesperson""")

for x in cursor:
    print(x)

    SPID SiteID   Spfname     Splname   StartDate    TermDate
0      1     SL    Archie      Dinesh  2021-01-01        NULL
1      2     SL  Margaret       Jason  2021-01-01  2021-04-08
2      3     SL    Maryla    Kunibert  2021-01-01        NULL
3      4     BO   Joachim       Gauri  2021-01-01  2021-02-14
4      5     BO    Kymani       Melia  2021-01-01  2021-09-08
5      6     BO   Roksana     Antonia  2021-01-01        NULL
6      7     BO    Shayan      Hewitt  2021-03-08        NULL
7      8     SL   Milburn       Guido  2021-05-12        NULL
8      9     PH     Robby        Lexy  2021-06-13        NULL
9     10     PH     Trudy       Allen  2021-06-15  2021-07-30
10    11     PH     Andre       Clare  2021-07-09        NULL
11    12     SL   Alberto       Dimas  2021-08-15        NULL
12    13     PH    Bethel      Derick  2021-08-18        NULL
13    14     BO     Kelan      Fowler  2021-11-13        NULL
14    15     PH      Miah  Wittington  2021-12-09        NULL
(1, 'SL'

In [25]:
# The Order Table
# Retain the orderid that you created


#df = pd.read_csv('order.csv')
infile = open('salesorder.csv','r')

firstline = 1

for x in infile:
    if (firstline == 1):
        firstline = 0
    else:
        y = x.split(',')
        
        satis = 'NULL'
        deliv = 'NULL'
        
        if (len(y[5]) > 0):
            satis = y[5]
        try:
            y[6] = int(y[6])
            deliv = y[6]
        except:
            pass
        
# This is for dates that might or might not be null.
#        if len(y[4]) > 1: 
#            y[4] = "'" + y[4] + "'"
#        else:
#            y[4] = 'NULL' # THIS PART IS IMPORTANT!!

        query = """INSERT INTO SalesOrder (orderid, custid, 
                        sellerid, odate, ototal, ordsatisfac, delsatisfac)
                   VALUES (%s,%s,%s,'%s',%s,%s,%s);""" % (y[0],y[1],y[2],y[3],y[4],satis,deliv)
           

        cursor.execute(query)
        
        conn.commit()
        
infile.close()

cursor.execute("SELECT * FROM salesorder;")
for x in cursor:
    print(x)

(28448, 3312, 11, datetime.date(2022, 1, 1), Decimal('88.90'), None, None)
(28449, 3313, 7, datetime.date(2022, 1, 1), Decimal('25083.90'), 3, 5)
(28450, 3314, 8, datetime.date(2022, 1, 1), Decimal('26504.80'), 5, 5)
(28451, 3315, 3, datetime.date(2022, 1, 1), Decimal('19439.00'), 4, 5)
(28452, 3316, 1, datetime.date(2022, 1, 1), Decimal('17995.00'), 5, 5)
(28453, 3317, 1, datetime.date(2022, 1, 1), Decimal('14.85'), None, None)
(28454, 3318, 6, datetime.date(2022, 1, 1), Decimal('26227.00'), 5, 4)
(28455, 3319, 1, datetime.date(2022, 1, 1), Decimal('18782.90'), 2, 5)
(28456, 3320, 11, datetime.date(2022, 1, 1), Decimal('18444.00'), 5, 5)
(28457, 3321, 13, datetime.date(2022, 1, 1), Decimal('24.75'), None, None)
(28458, 3322, 12, datetime.date(2022, 1, 1), Decimal('30763.00'), 5, 4)
(28459, 3323, 14, datetime.date(2022, 1, 1), Decimal('79.00'), None, None)
(28460, 3324, 6, datetime.date(2022, 1, 1), Decimal('27703.80'), 5, 5)
(28461, 3325, 8, datetime.date(2022, 1, 2), Decimal('30064.0

In [ ]:
# The OrderLine Table
# Auto-generate the PK (olid)


infile = open('oline.csv','r')

firstline = 1

for x in infile:
    if (firstline == 1):
        firstline = 0
    else:

        y = x.strip().split(',')
        
        bdate = 'NULL'
        pdate = 'NULL'
        ddate = 'NULL'
        
        y.append('NULL')
        y.append('NULL')
        y.append('NULL')
        
        if (len(y[6]) > 0 and y[6] != 'NULL'):
            bdate = "'" + y[6] + "'"
        if (len(y[7]) > 0 and y[7] != 'NULL'):
            pdate = "'" + y[7] + "'"
        if (len(y[8]) > 0 and y[8] != 'NULL'):
            ddate = "'" + y[8] + "'"

        query = """INSERT INTO Oline (orderid, productid, lineqty, lineprice, linetotal, builtdate, promisedate, deldate)
                   VALUES (%s,%s,%s,%s,%s,%s,%s,%s);""" % (y[1],y[2],y[3],y[4],y[5],bdate,pdate,ddate)
           

#        print ("OrderID: %s, CustID: %s, SellerID: %s, Odate: '%s', Ototal: %s" % (y[0],y[1],y[2],y[3],y[4]))
#        print(query)
    
        cursor.execute(query)
        
        conn.commit()
        
infile.close()

cursor.execute("SELECT * FROM oline")
for x in cursor:
    print(x)

In [ ]:
# The Customer Table
# Use the PK from the table!

infile = open('customer.csv','r')

header = 1
for x in infile:
    if header == 1:
        header = 0
    else:
        x = x.replace("'","")
        y = x.strip().split(',')
        ct = 0
        for z in y:
            if z == '' and ct in (11, 12, 13):
                y[ct] = 'z'
            elif z == '' and ct == 5:
                y[ct] = 'zz'
            elif z == '' and ct == 10:
                y[ct] = -1
            elif z == '':
                y[ct] = 'NULL'
            ct += 1
            
        
        cursor.execute("""INSERT INTO customer
                    (custid, custfname, custlname, custphone, custemail,custstate,
                    contactpref,pmtid,addedstamp,updatedstamp,hhi,
                    ismarried,haskids,travelswpet,pronoun)
                    VALUES (%s, '%s', '%s', '%s', '%s', '%s', '%s',
                    '%s', '%s', NULL, %s, '%s', '%s', '%s', '%s')"""
                  % (y[0],y[1],y[2],y[3],y[4],y[5],y[6],y[7],y[8],y[10],y[11],y[12],y[13],y[14]))

        conn.commit()

'''
for x in df.index:

    
    
query = 'SELECT * FROM customer;'

df = pd.read_sql(query, conn)
print(df)
'''

In [ ]:
# A little cleanup to deal with some malformed values in the source data.

mylist = ['custstate','ismarried','haskids','travelswpet','pronoun',
         'custphone','custemail','contactpref','pmtid']

for a in mylist:
    cursor.execute("""UPDATE customer
                SET %s = NULL
                WHERE %s = 'z' or %s = 'zz' or %s = 'NULL'""" % (a,a,a,a))
    
    conn.commit()
    
cursor.execute("""UPDATE customer
                SET hhi = NULL
                WHERE hhi < 0;""")
    


In [ ]:
# Seeing what ended up in Customer.

query = 'SELECT * FROM customer ORDER BY custid;'

df = pd.read_sql(query, conn)
print(df)

In [ ]:
# The Product Table
# Use the PK from the table!

infile = open('product.csv','r')

firstline = 1

for x in infile:
    if (firstline == 1):
        firstline = 0
    else:

        y = x.strip().split(',')
        
        catid = 'NULL'
        srcid = 'NULL'
        msrp = 'NULL'
        leadtime = 'NULL'
        qty = 'NULL'
        rolev = 'NULL'
        enddate = 'NULL'
        
        y.append('NULL')
        y.append('NULL')
        
        
        if (len(y[1]) > 0):
            catid = int(y[1])
        if (len(y[2]) > 0):
            srcid = int(y[2])
        if (len(y[4]) > 0):
            msrp = y[4]
        if (len(y[6]) > 0):
            leadtime = y[6]
        if (len(y[7]) > 0):
            qty = int(y[7])
        if (len(y[8]) > 0):
            rolev = int(y[8])
        if len(y[10]) > 0:
            enddate = "'" + y[10] + "'"

        query = """INSERT INTO Product (prodid, catid, sourceid, prodname, msrp, proddesc,
                                        leadtime, qtyoh, enddate, startdate, rolevel)
                    VALUES (%s,%s,%s,'%s',%s,'%s',%s,%s,%s,'%s',%s)""" % (y[0], catid,srcid, y[3],msrp,
                                                                          y[5],leadtime, qty,enddate,y[9],
                                                                          rolev)
                    
#        print(query)
    
        cursor.execute(query)
        
        conn.commit()
        
infile.close()

In [ ]:
# Just checking out the Product table for some reason.

query = 'SELECT * FROM product ORDER BY prodid;'

df = pd.read_sql(query, conn)
print(df)

In [ ]:
# The Vendor Table
# Auto-generate the PK (vendorID)

infile = open('vendor.csv','r')

firstline = 1

for x in infile:
    if (firstline == 1):
        firstline = 0
    else:
        y = x.strip().split(',')
        
        query = """INSERT INTO Vendor (vendorname, contactname)
                    VALUES ('%s','%s')""" % (y[0],y[1])
                    
        #print(query)
    
        cursor.execute(query)
        
        conn.commit()
        
infile.close()

In [ ]:
query = 'SELECT * FROM vendor ORDER BY vendorid;'

df = pd.read_sql(query, conn)
print(df)

In [ ]:
# The Prodcat Table
# Auto-generate the PK (vendorID)


infile = open('prodcat.csv','r')

firstline = 1

for x in infile:
    if (firstline == 1):
        firstline = 0
    else:
        y = x.strip().split(',')
        
        query = """INSERT INTO prodcat (catid,category,catdesc)
                    VALUES (%s,'%s','%s')""" % (y[0],y[1],y[2])
                    
        print(query)
    
        cursor.execute(query)
        
        conn.commit()
        
infile.close()

query = 'SELECT * FROM prodcat ORDER BY catid;'

df = pd.read_sql(query, conn)
print(df)

In [ ]:
# The PriceMove Table
# Auto-generate the PK (priceid)


infile = open('pricelog.csv','r')

firstline = 1

for x in infile:
    if (firstline == 1):
        firstline = 0
    else:
        y = x.strip().split(',')
        
        query = """INSERT INTO pricelog (prodid,theprice,startdate,approver,approvedate)
                    VALUES (%d,%.2f,'%s','%s','%s')""" % (int(y[0]),float(y[1]),y[2],y[3],y[4])
                    
        #print(query)
    
        cursor.execute(query)
        
        conn.commit()
        
infile.close()

query = 'SELECT * FROM pricelog ORDER BY priceid;'

df = pd.read_sql(query, conn)
print(df)

In [ ]:
cursor.close()
conn.close()